In [1]:
import pandas as pd

In [2]:
def LoadAndFormatData(date_to_begin=None):

    # Load data into pandas dataframe
    df = pd.read_parquet('Data/BoQ_Data.parquet')
    df['DATE'] = pd.to_datetime(df['DATE'])
    if date_to_begin is not None:
        date_to_begin = pd.to_datetime(date_to_begin)
        df = df[df['DATE'] >= date_to_begin]

    def AddMarketReturn(dataframe):
        """Adds a market return variable (mkt) to the dataframe"""

        # Define a function to calculate the weighted mean return
        def weighted_mean(group):
            return (group['RET'] * group['ME']).sum() / group['ME'].sum()

        # Group by 'year' and apply the weighted mean to each group
        dataframe['mkt'] = dataframe.groupby('DATE').apply(
            lambda group: pd.Series(weighted_mean(group), index=group.index)
        ).reset_index(drop=True)

        return dataframe
    
    df['RET_t+1'] = df.groupby("PERMNO")['RET'].shift(-1)
    
    df = AddMarketReturn(df)
    return df

In [3]:
#LoadAndFormatData()